In [2]:
import sys
from pathlib import Path

import numpy as np
import sys
!{sys.executable} -m pip install xgboost
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.8 MB/s  0:00:00


In [5]:
print("Notebook location:", Path().resolve())
print("Current sys.path:", sys.path)

project_root = Path("/Users/arnaudthomas/code/bennystu/Trend-Surgeon/time-series-cross-validator")
sys.path.append(str(project_root))

Notebook location: /Users/arnaudthomas/code/bennystu/Trend-Surgeon/notebooks
Current sys.path: ['/Users/arnaudthomas/.pyenv/versions/3.10.6/lib/python310.zip', '/Users/arnaudthomas/.pyenv/versions/3.10.6/lib/python3.10', '/Users/arnaudthomas/.pyenv/versions/3.10.6/lib/python3.10/lib-dynload', '', '/Users/arnaudthomas/.pyenv/versions/3.10.6/envs/Trend-Surgeon/lib/python3.10/site-packages', '/Users/ArnaudThs/code/bennystu/Trend-Surgeon/time-series-cross-validator']


In [ ]:
import sys
!{sys.executable} -m pip install python-dotenv

In [9]:
from ts_boilerplate.generate_dummy_data import generate_time_series

X, y = generate_time_series(n_samples=500, input_length=10, n_features=5)
print(X.shape, y.shape)

(500, 10, 5) (500, 1)


In [10]:
X_flat = X.reshape(X.shape[0], -1)
print(X_flat.shape)

(500, 50)


In [11]:
X_train, X_val, y_train, y_val = train_test_split(
    X_flat, y, test_size=0.2, shuffle=False
)

In [12]:
learning_rates = [0.01, 0.1]
n_estimators_list = [100, 200]
max_depths = [3, 6]
subsamples = [0.8, 1.0]

In [ ]:
best_loss = float('inf')
best_config = None
best_model = None

for lr in learning_rates:
    for n_est in n_estimators_list:
        for depth in max_depths:
            for subs in subsamples:

                print(f"Training XGB | lr={lr} | n_estimators={n_est} | depth={depth} | subsample={subs}")

                model = XGBRegressor(
                    learning_rate=lr,
                    n_estimators=n_est,
                    max_depth=depth,
                    subsample=subs,
                    objective='reg:squarederror',
                    tree_method="hist"
                )

                model.fit(X_train, y_train)
                y_pred = model.predict(X_val)

                loss = mean_squared_error(y_val, y_pred)

                if loss < best_loss:
                    best_loss = loss
                    best_config = {
                        "learning_rate": lr,
                        "n_estimators": n_est,
                        "max_depth": depth,
                        "subsample": subs
                    }
                    best_model = model

print("\n✅ Best configuration:", best_config)
print("Best validation loss:", best_loss)

Training XGB | lr=0.01 | n_estimators=100 | depth=3 | subsample=0.8
Training XGB | lr=0.01 | n_estimators=100 | depth=3 | subsample=1.0
Training XGB | lr=0.01 | n_estimators=100 | depth=6 | subsample=0.8
Training XGB | lr=0.01 | n_estimators=100 | depth=6 | subsample=1.0
Training XGB | lr=0.01 | n_estimators=200 | depth=3 | subsample=0.8
Training XGB | lr=0.01 | n_estimators=200 | depth=3 | subsample=1.0
Training XGB | lr=0.01 | n_estimators=200 | depth=6 | subsample=0.8
Training XGB | lr=0.01 | n_estimators=200 | depth=6 | subsample=1.0
Training XGB | lr=0.1 | n_estimators=100 | depth=3 | subsample=0.8
Training XGB | lr=0.1 | n_estimators=100 | depth=3 | subsample=1.0
Training XGB | lr=0.1 | n_estimators=100 | depth=6 | subsample=0.8
Training XGB | lr=0.1 | n_estimators=100 | depth=6 | subsample=1.0
Training XGB | lr=0.1 | n_estimators=200 | depth=3 | subsample=0.8
Training XGB | lr=0.1 | n_estimators=200 | depth=3 | subsample=1.0
Training XGB | lr=0.1 | n_estimators=200 | depth=6 | s

In [14]:
y_pred = best_model.predict(X_flat[:5])
print("First 5 predictions:\n", y_pred)

First 5 predictions:
 [1.0011089  0.9980069  0.9919952  1.0038395  0.99741775]
